# Implémentation de FastText

## Import de librairies

In [ ]:
import os
import re
import itertools

import fasttext
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from constants import TRAINING_DATA_PATH, TEXT_COLUMN, LABEL_COLUMN, ALPHABETS_FT

ALPHABETS = ALPHABETS_FT


## Fonctions utiles

In [2]:
def prepare_fasttext_data(data, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for i in range(len(data)):
            text = data.iloc[i][TEXT_COLUMN]
            lang = data.iloc[i][LABEL_COLUMN]
            # FastText attend le format: __label__LANG texte
            f.write(f"__label__{lang} {text}\n")            

In [50]:
def detect_alphabet(text):
    """
    Détecte l'alphabet principal utilisé dans un texte en analysant la fréquence des caractères.
    Renvoie "Inconnu" si aucun alphabet connu n'est dominant.
    """
    text = ''.join(c for c in text if not c.isspace() and c not in '.,;:!?-()[]{}\'\"')
    
    if not text:
        return "Inconnu", 0.0

    counts = {name: 0 for name in ALPHABETS}
    total_chars = len(text)

    # Comptage des caractères par alphabet
    for char in text:
        char_code = ord(char)
        for name, ranges in ALPHABETS.items():
            if any(start <= char_code <= end for start, end in ranges):
                counts[name] += 1
                break  # Un caractère ne peut appartenir qu'à un seul alphabet

    # Trouver l'alphabet le plus représenté
    best_match = max(counts.items(), key=lambda x: x[1])
    alphabet_name, max_count = best_match
    percentage = (max_count / total_chars) * 100

    # Si l'alphabet dominant représente moins de 50% du texte, on retourne "Inconnu"
    if percentage < 50:
        return "Inconnu", 0.0

    return alphabet_name, percentage

# Test de detection d'alphabets avec input

# input_text = input("Entrez un texte à analyser: ")
# detected, percentage = detect_alphabet(input_text)
# print(f"Texte: {input_text[:20]}...")
# print(f"Détecté: {detected} ({percentage:.1f}%)")
    

## Training

In [57]:
# Extraction des données
data = pd.read_csv(TRAINING_DATA_PATH)

# Prise en compte du label nan (qui est le code d'une langue)
data = data.fillna("nana")

# Nettoyage des textes
def clean_text(text):
    text = text.lower()  # Convertir en minuscule
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()  # Supprimer espaces multiples
    return text

data[TEXT_COLUMN] = data[TEXT_COLUMN].apply(clean_text)

# Nettoyage des données
print("Données sans label : ", data[LABEL_COLUMN].isna().sum())
data = data.dropna(subset=[LABEL_COLUMN])

# Ajouter une colonne pour l'alphabet détecté
data['alphabet'], data['alphabet_percentage'] = zip(*data[TEXT_COLUMN].map(detect_alphabet))

# Répartition des alphabets détectés, donné le nombre de données et de labels pour chaque alphabet
alphabet_counts = data['alphabet'].value_counts()
# Ajouter le nombre de labels par alphabet dans le meme dataframe alphabet counts
alphabet_counts = pd.concat([alphabet_counts, data.groupby('alphabet')[LABEL_COLUMN].nunique()], axis=1)
print(alphabet_counts)

# Vérification que toutes les données ont un alphabet détecté
print("Données sans alphabet", data['alphabet'].isna().sum())

# Grille d'hyperparamètres à tester

# Base param grid
# param_grid = {
#     "lr": [0.4, 0.5, 0.6],  # Autour de 0.5
#     "dim": [16, 20],  # Autour de 16
#     "epoch": [50, 60],  # Autour de 50
#     "minn": [2, 3],  # Autour de 2
#     "maxn": [5, 6],  # Autour de 6
#     "wordNgrams": [1],  # Pas de modification
#     "minCount": [1],  # Pas de modification
#     "neg": [5, 6],  # Autour de 5
# }

param_grid_global = {
    "Inconnu": {
        "lr": [0.5],
        "dim": [16],
        "epoch": [100],
        "minn": [3],
        "maxn": [5],
        "wordNgrams": [1],
        "minCount": [1],
        "neg": [5],
    },
    "Cyrillique": {
        "lr": [0.2],
        "dim": [30],
        "epoch": [110],
        "minn": [3],
        "maxn": [4],
        "wordNgrams": [1],
        "minCount": [1],
        "neg": [4],
    },
    "Latin": {
        "lr": [0.4],
        "dim": [40],
        "epoch": [40],
        "minn": [3],
        "maxn": [7],
        "wordNgrams": [1],
        "minCount": [1],
        "neg": [10],
    },
    "Arabe": {
        "lr": [0.55],
        "dim": [100],
        "epoch": [110],
        "minn": [3],
        "maxn": [5],
        "wordNgrams": [1],
        "minCount": [1],
        "neg": [8],
    }
}

# Créer le dossier pour les données temporaires si nécessaire
os.makedirs(f"data/alphabet", exist_ok=True)
os.makedirs(f"models", exist_ok=True)

# Variables globales pour les métriques finales
global_instances = 0
global_right_predictions = 0
global_results = []

# Boucle sur chaque alphabet (choisir les alphabets que l'on veut traiter)
for alphabet in ["Cyrillique", "Latin", "Arabe", "Inconnu"]:
    print("-" * 50)
    print(f"Traitement de l'alphabet: {alphabet}")
    
    # Filtrer les données pour l'alphabet actuel
    alphabet_data = data[data['alphabet'] == alphabet]

    # Enlever les instances qui sont les seuls représentants de leur langue
    label_counts = alphabet_data[LABEL_COLUMN].value_counts()
    single_instances = label_counts[label_counts == 1].index
    alphabet_data = alphabet_data[~alphabet_data[LABEL_COLUMN].isin(single_instances)]
    
    
    # Afficher le nombre d'instance pour chaque label
    print("Nombre d'instances par langue:")
    print(alphabet_data[LABEL_COLUMN].value_counts())
    
    # 1. SPLIT INITIAL TRAIN/TEST
    train_data, test_data = train_test_split(
        alphabet_data, 
        test_size=0.20, 
        shuffle=True, 
        random_state=10, 
        stratify=alphabet_data[LABEL_COLUMN]
    )

    print(f"Split initial train/test pour {alphabet}: {len(train_data)} instances train, {len(test_data)} instances test")
    
    
    # Variables pour stocker les résultats de la validation croisée
    best_params = None
    best_model = None
    best_test_accuracy = 0
    
    # Générer toutes les combinaisons d'hyperparamètres
    all_param_combinations = [dict(zip(param_grid_global[alphabet].keys(), params)) 
                             for params in itertools.product(*param_grid_global[alphabet].values())]
    
    print(f"Évaluation de {len(all_param_combinations)} combinaisons d'hyperparamètres")
    
    # Prépare les données pour FastText
    train_file = f"data/alphabet/train_{alphabet}.txt"
    test_file = f"data/alphabet/test_{alphabet}.txt"

    prepare_fasttext_data(data = train_data, output_file=train_file)
    prepare_fasttext_data(data = test_data, output_file=test_file)
    
    # Itérer d'abord sur les combinaisons d'hyperparamètres
    for param_idx, param_combination in enumerate(all_param_combinations):
        print(f"Combinaison {param_idx+1}/{len(all_param_combinations)}: {param_combination}")
    

        model = fasttext.train_supervised(
            input=train_file,
            lr=param_combination['lr'],
            dim=param_combination['dim'],
            epoch=param_combination['epoch'],
            wordNgrams=param_combination['wordNgrams'],
            minCount=param_combination['minCount'],
            minn=param_combination['minn'],
            maxn=param_combination['maxn'],
            neg=param_combination['neg'],
        )
        
        # Évaluer le modèle sur les données de validation
        test_result = model.test(test_file)
        test_accuracy = test_result[1]
        
        train_result = model.test(train_file)
        train_accuracy = train_result[1]
        
        # Conserver les meilleurs hyperparamètres
        if test_accuracy > best_test_accuracy:
            best_test_accuracy = test_accuracy
            best_train_accuracy = train_accuracy
            best_params = param_combination
            best_model = model
            print(f"  Nouvelle meilleure combinaison trouvée!")
            print(f"  Précision: {test_accuracy:.4f}")
            

    # Afficher les meilleurs hyperparamètres trouvés
    print(f"\nRésultats de la recherche d'hyperparamètres pour {alphabet}:")
    print(f"Meilleure précision : {best_test_accuracy:.4f}")
    print(f"Meilleurs hyperparamètres trouvés: {best_params}")
    
    # Mettre à jour les compteurs globaux
    nb_test_samples = len(test_data)
    nb_right_predictions = int(best_test_accuracy * nb_test_samples)
    print(f"Nombre d'instances dans le set de test: {nb_test_samples}")
    print(f"Nombre de prédictions correctes: {nb_right_predictions}")
    
    # Afficher une matrice de confusion du set de test
    # Obtenir la liste des labels uniques et créer un mapping label → index
    unique_labels = sorted(test_data[LABEL_COLUMN].unique())  
    label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
    
    global_instances += nb_test_samples
    global_right_predictions += nb_right_predictions
    
    # Enregistrer les résultats pour cet alphabet
    global_results.append({
        'alphabet': alphabet,
        'test_accuracy': best_test_accuracy,
        'train_accuracy': best_train_accuracy,
        'hyperparameters': best_params,
        'test_samples': nb_test_samples,
        'correct_predictions': nb_right_predictions
    })
    
    # Sauvegarder le modèle final
    best_model.save_model(f"models/model_{alphabet}.bin")
    print(f"Modèle final sauvegardé: models/model_{alphabet}.bin")

# Calculer la précision globale sur tous les alphabets
global_accuracy = global_right_predictions / global_instances if global_instances > 0 else 0

# Afficher les résultats globaux
print("\n" + "=" * 50)
print("RÉSULTATS GLOBAUX SUR TOUS LES ALPHABETS")
print(f"Précision globale: {global_accuracy:.4f}")
print(f"Nombre total d'échantillons testés: {global_instances}")
print(f"Nombre total de prédictions correctes: {global_right_predictions}")

# Afficher les résultats détaillés par alphabet
print("\nRésultats détaillés par alphabet:")
for result in sorted(global_results, key=lambda x: x['test_accuracy'], reverse=True):
    print(f"Alphabet: {result['alphabet']}")
    print(f"  Précision test : {result['test_accuracy']:.4f}")
    print(f"  Précision train: {result['train_accuracy']:.4f}")
    print(f"  Hyperparamètres: {result['hyperparameters']}")
    print(f"  Nombre d'échantillons testés: {result['test_samples']}")
    print(f"  Nombre de prédictions correctes: {result['correct_predictions']}")
    print("-" * 50)

Données sans label :  0
            count  Label
alphabet                
Latin       28205    303
Cyrillique   3299     34
Arabe        2470     25
Inconnu      2233     54
Devanagari   1191     12
Chinois       499      6
Hébreu        235      3
Géorgien      199      2
Grec          199      2
Gujarati      100      1
Thaï          100      1
Coréen        100      1
Katakana       12      1
Hiragana       12      1
Données sans alphabet 0
--------------------------------------------------
Traitement de l'alphabet: Cyrillique
Nombre d'instances par langue:
Label
myv    100
hbs    100
kbd    100
rue    100
uzb    100
bel    100
abk    100
chv    100
kaa    100
tyv    100
bak    100
sah    100
tat    100
alt    100
krc    100
kaz    100
crh    100
ukr    100
mon    100
udm    100
kjh    100
kom    100
mhr    100
oss    100
kir    100
srp    100
mkd    100
bul    100
uzn    100
tgk    100
bew    100
che    100
tuk     98
Name: count, dtype: int64
Split initial train/test pour Cyrilliq

Read 0M words
Number of words:  34071
Number of labels: 33
Progress: 100.0% words/sec/thread:  754496 lr:  0.000000 avg.loss:  0.632085 ETA:   0h 0m 0s


  Nouvelle meilleure combinaison trouvée!
  Précision: 0.8424

Résultats de la recherche d'hyperparamètres pour Cyrillique:
Meilleure précision : 0.8424
Meilleurs hyperparamètres trouvés: {'lr': 0.2, 'dim': 30, 'epoch': 110, 'minn': 3, 'maxn': 4, 'wordNgrams': 1, 'minCount': 1, 'neg': 4}
Nombre d'instances dans le set de test: 660
Nombre de prédictions correctes: 556
Modèle final sauvegardé: models/model_Cyrillique.bin
--------------------------------------------------
Traitement de l'alphabet: Latin
Nombre d'instances par langue:
Label
arg    100
mgh    100
luo    100
cab    100
nnb    100
      ... 
miq      3
crs      2
tvl      2
pau      2
gil      2
Name: count, Length: 299, dtype: int64
Split initial train/test pour Latin: 22560 instances train, 5641 instances test
Évaluation de 1 combinaisons d'hyperparamètres
Combinaison 1/1: {'lr': 0.4, 'dim': 40, 'epoch': 40, 'minn': 3, 'maxn': 7, 'wordNgrams': 1, 'minCount': 1, 'neg': 10}


Read 0M words
Number of words:  210698
Number of labels: 299
Progress: 100.0% words/sec/thread:  248929 lr:  0.000000 avg.loss:  1.307206 ETA:   0h 0m 0s


  Nouvelle meilleure combinaison trouvée!
  Précision: 0.7653

Résultats de la recherche d'hyperparamètres pour Latin:
Meilleure précision : 0.7653
Meilleurs hyperparamètres trouvés: {'lr': 0.4, 'dim': 40, 'epoch': 40, 'minn': 3, 'maxn': 7, 'wordNgrams': 1, 'minCount': 1, 'neg': 10}
Nombre d'instances dans le set de test: 5641
Nombre de prédictions correctes: 4317
Modèle final sauvegardé: models/model_Latin.bin
--------------------------------------------------
Traitement de l'alphabet: Arabe
Nombre d'instances par langue:
Label
pus    100
kur    100
ara    100
mzn    100
arz    100
pes    100
tgk    100
uig    100
apc    100
urd    100
azb    100
arb    100
fas    100
prs    100
snd    100
glk    100
pnb    100
ckb    100
som     99
afb     98
ary     97
acm     97
aze     97
ajp     97
hau     85
Name: count, dtype: int64
Split initial train/test pour Arabe: 1976 instances train, 494 instances test
Évaluation de 1 combinaisons d'hyperparamètres
Combinaison 1/1: {'lr': 0.55, 'dim': 10

Read 0M words
Number of words:  21580
Number of labels: 25
Progress: 100.0% words/sec/thread:  307579 lr:  0.000000 avg.loss:  0.266341 ETA:   0h 0m 0s


  Nouvelle meilleure combinaison trouvée!
  Précision: 0.5972

Résultats de la recherche d'hyperparamètres pour Arabe:
Meilleure précision : 0.5972
Meilleurs hyperparamètres trouvés: {'lr': 0.55, 'dim': 100, 'epoch': 110, 'minn': 3, 'maxn': 5, 'wordNgrams': 1, 'minCount': 1, 'neg': 8}
Nombre d'instances dans le set de test: 494
Nombre de prédictions correctes: 295
Modèle final sauvegardé: models/model_Arabe.bin
--------------------------------------------------
Traitement de l'alphabet: Inconnu
Nombre d'instances par langue:
Label
amh     100
mal     100
asm     100
mya     100
sat     100
sin     100
bod     100
div     100
tam     100
hye     100
tir     100
pan     100
lao     100
ori     100
kan     100
bpy     100
ory     100
iku     100
dzo     100
khm     100
jpn      64
tbz      31
hyw      25
hau      15
ksw      13
kat       9
guj       8
fon       6
yue       6
bqc       6
nana      5
wuu       4
uig       4
ajp       3
ary       3
aze       3
mon       3
acm       3
tat    

Read 0M words
Number of words:  28091
Number of labels: 43
Progress: 100.0% words/sec/thread:  809794 lr:  0.000000 avg.loss:  0.440972 ETA:   0h 0m 0s


  Nouvelle meilleure combinaison trouvée!
  Précision: 0.9011

Résultats de la recherche d'hyperparamètres pour Inconnu:
Meilleure précision : 0.9011
Meilleurs hyperparamètres trouvés: {'lr': 0.5, 'dim': 16, 'epoch': 100, 'minn': 3, 'maxn': 5, 'wordNgrams': 1, 'minCount': 1, 'neg': 5}
Nombre d'instances dans le set de test: 445
Nombre de prédictions correctes: 401
Modèle final sauvegardé: models/model_Inconnu.bin

RÉSULTATS GLOBAUX SUR TOUS LES ALPHABETS
Précision globale: 0.7692
Nombre total d'échantillons testés: 7240
Nombre total de prédictions correctes: 5569

Résultats détaillés par alphabet:
Alphabet: Inconnu
  Précision test : 0.9011
  Précision train: 0.9848
  Hyperparamètres: {'lr': 0.5, 'dim': 16, 'epoch': 100, 'minn': 3, 'maxn': 5, 'wordNgrams': 1, 'minCount': 1, 'neg': 5}
  Nombre d'échantillons testés: 445
  Nombre de prédictions correctes: 401
--------------------------------------------------
Alphabet: Cyrillique
  Précision test : 0.8424
  Précision train: 0.9996
  Hype